In [2]:
import numpy as np
import pandas as pd

In [ ]:
netflix_data = pd.read_csv("netflix_titles.csv")
imdb_data = pd.read_csv("title_basic_imdb.tsv", sep = '\t')

In [ ]:
imdb_rating = pd.read_csv("movie_ratings.tsv", sep = "\t")

In [ ]:
new_dataframe_with_rating = pd.merge(imdb_data.set_index('tconst'), imdb_rating.set_index('tconst'), left_index=True, right_index=True).drop_duplicates()

In [ ]:
new_dataframe_with_rating

In [ ]:
new_dataframe_with_rating = new_dataframe_with_rating[new_dataframe_with_rating['titleType']!='tvEpisode']

In [ ]:
netflix_data['title'] = netflix_data['title'].str.lower()
new_dataframe_with_rating['primaryTitle'] = new_dataframe_with_rating['primaryTitle'].str.lower()
new_dataframe_with_rating = new_dataframe_with_rating.rename(columns={"primaryTitle":"title"})

In [ ]:
new_dataframe_with_rating = new_dataframe_with_rating[new_dataframe_with_rating.startYear.apply(lambda x: str(x).isnumeric())]
new_dataframe_with_rating['startYear'] = new_dataframe_with_rating['startYear'].astype(int)
new_dataframe_with_rating = new_dataframe_with_rating.rename(columns={"startYear":"year"})

In [ ]:
new_data_netflix = pd.merge(netflix_data,new_dataframe_with_rating, left_on = ['title','release_year'], right_on = ['title', 'year'])

In [ ]:
reduced_data = new_data_netflix.drop(columns=['titleType', 'date_added', 'description','runtimeMinutes','isAdult','endYear'])
reduced_data.head()

In [ ]:
reduced_data = reduced_data.drop(columns=['country', 'listed_in', 'release_year'])

In [5]:
# >8: critical acclaimed
# 6-<8: positive reviewed
# 4-<6: medium
# <4: negative

def get_rank(score):
    if score['averageRating'] > 8:
        rank = "Critical Acclaimed"
    elif score['averageRating'] >= 6 and score['averageRating']  <= 8:
        rank = "Positive"
    elif score['averageRating'] >= 4 and score['averageRating']  <= 6:
        rank = "Medium"
    else:
        rank = "Negative"
    return rank

reduced_data['ranking'] = reduced_data.apply(get_rank, axis=1 )
reduced_data

,show_id,type,title,director,cast,rating,duration,originalTitle,year,genres,averageRating,numVotes,ranking
0,s1,Movie,dick johnson is dead,Kirsten Johnson,NaN,PG-13,90 min,Dick Johnson Is Dead,2020,"Biography,Documentary,Drama",7.4,6322,Positive
1,s3,TV Show,ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",TV-MA,1 Season,Braqueurs,2021,"Action,Crime,Drama",7.0,2399,Positive
2,s4,TV Show,jailbirds new orleans,NaN,NaN,TV-MA,1 Season,Jailbirds New Orleans,2021,"Documentary,Reality-TV",6.6,198,Positive
3,s6,TV Show,midnight mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",TV-MA,1 Season,Midnight Mass,2021,"Drama,Fantasy,Horror",7.7,100194,Positive
4,s7,Movie,my little pony: a new generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",PG,91 min,My Little Pony: A New Generation,2021,"Adventure,Animation,Comedy",6.9,3355,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6415,s8801,TV Show,zindagi gulzar hai,NaN,"Sanam Saeed, Fawad Khan, Ayesha Omer, Mehreen ...",TV-PG,1 Season,Zindagi Gulzar Hai,2012,Romance,9.0,3489,Critical Acclaimed
6416,s8803,Movie,zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",R,158 min,Zodiac,2007,"Crime,Drama,Mystery",7.7,520666,Positive
6417,s8805,Movie,zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",R,88 min,Zombieland,2009,"Action,Comedy,Horror",7.6,557805,Positive
6418,s8806,Movie,zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",PG,88 min,Zoom,2006,"Action,Adventure,Comedy",4.3,20879,Medium


In [ ]:
reduced_data.to_csv('netflix_ratings_data.csv', sep=',', index=False)
